In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
df = pd.read_csv("/content/2011-Q1-cabi-trip-history-data.csv")
df.head(10)

Duration       Start date         End date  \
0   0h 1min. 50sec.  3/31/2011 23:58    4/1/2011 0:00   
1  0h 16min. 21sec.  3/31/2011 23:52    4/1/2011 0:08   
2   0h 3min. 19sec.  3/31/2011 23:47  3/31/2011 23:50   
3   0h 5min. 44sec.  3/31/2011 23:45  3/31/2011 23:50   
4   0h 8min. 42sec.  3/31/2011 23:34  3/31/2011 23:43   
5  0h 19min. 57sec.  3/31/2011 23:32  3/31/2011 23:52   
6   0h 7min. 51sec.  3/31/2011 23:31  3/31/2011 23:39   
7   0h 2min. 56sec.  3/31/2011 23:30  3/31/2011 23:33   
8   0h 9min. 55sec.  3/31/2011 23:29  3/31/2011 23:39   
9   0h 17min. 6sec.  3/31/2011 23:28  3/31/2011 23:45   

                                       Start station  \
0                       14th & Harvard St NW (31105)   
1                             19th & L St NW (31224)   
2  Lincoln Park / 13th & East Capitol St NE  (31619)   
3                             14th & R St NW (31202)   
4                    Park Rd & Holmead Pl NW (31602)   
5              4th St & Massachusetts Ave NW (31604)   
6     37th & O St NW / Georgetown University (31236)   
7                       14th & Harvard St NW (31105)   
8                    Lamont & Mt Pleasant NW (31107)   
9                  13th St & New York Ave NW (31227)   

                                    End station   Bike# Member Type  
0                  16th & Harvard St NW (31103)  W00749  Registered  
1     7th & Water St SW / SW Waterfront (31609)  W01048      Casual  
2                        13th & H St NE (31611)  W00340  Registered  
3  Massachusetts Ave & Dupont Circle NW (31200)  W00981  Registered  
4                        15th & P St NW (31201)  W00449  Registered  
5           Adams Mill & Columbia Rd NW (31104)  w00583  Registered  
6                        21st & M St NW (31212)  W00318  Registered  
7                        14th & V St NW (31101)  W00577  Registered  
8                         7th & T St NW (31109)  W00416  Registered  
9        Georgia Ave and Fairmont St NW (31207)  W00903  Registered

In [3]:
df['Duration'] = df['Duration'].str.replace('min', '')
df['Duration'] = df['Duration'].str.replace('sec', '')
df['Duration'] = df['Duration'].str.replace('h', '')
df['Duration'] = df['Duration'].str.replace(' ', '')
df['Duration'] = df['Duration'].str.replace('.', '').astype(float)

<ipython-input-3-c1f639b5b026>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Duration'] = df['Duration'].str.replace('.', '').astype(float)


In [4]:
address=df['End station'].unique()
print(address)
df = df.dropna()

['16th & Harvard St NW (31103)'
 '7th & Water St SW / SW Waterfront (31609)' '13th & H St NE (31611)'
 'Massachusetts Ave & Dupont Circle NW (31200)' '15th & P St NW (31201)'
 'Adams Mill & Columbia Rd NW (31104)' '21st & M St NW (31212)'
 '14th & V St NW (31101)' '7th & T St NW (31109)'
 'Georgia Ave and Fairmont St NW (31207)' '20th & E St NW (31204)'
 '14th St & Spring Rd NW (31401)' '14th & Rhode Island Ave NW (31203)'
 '17th & Corcoran St NW (31214)' '25th St & Pennsylvania Ave NW (31237)'
 '10th & Monroe St NE (31504)' 'C & O Canal & Wisconsin Ave NW (31225)'
 '18th & M St NW (31221)' '11th & Kenyon St NW (31102)'
 '3rd & H St NE (31616)' 'Park Rd & Holmead Pl NW (31602)'
 'Calvert St & Woodley Pl NW (31106)'
 'Georgia & New Hampshire Ave NW (31400)' '10th & U St NW (31111)'
 'Potomac & Pennsylvania Ave SE (31606)'
 'Columbus Circle / Union Station (31623)' 'Florida Ave & R St NW (31503)'
 '14th & D St SE (31607)' 'McPherson Square - 14th & H St NW (31216)'
 'Lincoln Park / 13th 

In [5]:
from sklearn.preprocessing import LabelEncoder
gle = LabelEncoder()
address_labels = gle.fit_transform(df['End station'])
address_mappings = {index: label for index, label in enumerate(gle.classes_)}
address_mappings

{0: '10th & Monroe St NE (31504)',
 1: '10th & U St NW (31111)',
 2: '10th St & Constitution Ave NW (31219)',
 3: '11th & Kenyon St NW (31102)',
 4: '12th & Army Navy Dr (31008)',
 5: '12th & Hayes St (31001)',
 6: '12th & Newton St NE (31501)',
 7: '13th & D St NE (31622)',
 8: '13th & H St NE (31611)',
 9: '13th St & New York Ave NW (31227)',
 10: '14th & D St SE (31607)',
 11: '14th & G St NW (31238)',
 12: '14th & Harvard St NW (31105)',
 13: '14th & R St NW (31202)',
 14: '14th & Rhode Island Ave NW (31203)',
 15: '14th & V St NW (31101)',
 16: '14th St & Spring Rd NW (31401)',
 17: '14th St Heights / 14th & Crittenden St NW (31402)',
 18: '15th & Crystal Dr (31003)',
 19: '15th & Hayes St (31005)',
 20: '15th & P St NW (31201)',
 21: '16th & Harvard St NW (31103)',
 22: '16th & U St NW (31229)',
 23: '17th & Corcoran St NW (31214)',
 24: '17th & K St NW [formerly 17th & L St NW] (31213)',
 25: '18th & Bell St (31007)',
 26: '18th & Hayes St (31004)',
 27: '18th & M St NW (31221)'

In [6]:
df['End station'] = address_labels

In [7]:
gle = LabelEncoder()
address_labels = gle.fit_transform(df['Start station'])
address_mappings = {index : label for index, label in enumerate(gle.classes_)}
address_mappings

{0: '10th & Monroe St NE (31504)',
 1: '10th & U St NW (31111)',
 2: '10th St & Constitution Ave NW (31219)',
 3: '11th & Kenyon St NW (31102)',
 4: '12th & Army Navy Dr (31008)',
 5: '12th & Hayes St (31001)',
 6: '12th & Newton St NE (31501)',
 7: '13th & D St NE (31622)',
 8: '13th & H St NE (31611)',
 9: '13th St & New York Ave NW (31227)',
 10: '14th & D St SE (31607)',
 11: '14th & G St NW (31238)',
 12: '14th & Harvard St NW (31105)',
 13: '14th & R St NW (31202)',
 14: '14th & Rhode Island Ave NW (31203)',
 15: '14th & V St NW (31101)',
 16: '14th St & Spring Rd NW (31401)',
 17: '14th St Heights / 14th & Crittenden St NW (31402)',
 18: '15th & Crystal Dr (31003)',
 19: '15th & Hayes St (31005)',
 20: '15th & P St NW (31201)',
 21: '16th & Harvard St NW (31103)',
 22: '16th & U St NW (31229)',
 23: '17th & Corcoran St NW (31214)',
 24: '17th & K St NW [formerly 17th & L St NW] (31213)',
 25: '18th & Bell St (31007)',
 26: '18th & Hayes St (31004)',
 27: '18th & M St NW (31221)'

In [8]:
df['Start station'] = address_labels

In [9]:
df.head()

Duration       Start date         End date  Start station  End station  \
0     150.0  3/31/2011 23:58    4/1/2011 0:00             12           21   
1    1621.0  3/31/2011 23:52    4/1/2011 0:08             30           58   
2     319.0  3/31/2011 23:47  3/31/2011 23:50             86            8   
3     544.0  3/31/2011 23:45  3/31/2011 23:50             13           88   
4     842.0  3/31/2011 23:34  3/31/2011 23:43             94           20   

    Bike# Member Type  
0  W00749  Registered  
1  W01048      Casual  
2  W00340  Registered  
3  W00981  Registered  
4  W00449  Registered

In [10]:
gle = LabelEncoder()
bike_labels = gle.fit_transform(df['Bike#'])
bike_mappings = {index : label for index, label in enumerate(gle.classes_)}
bike_mappings

{0: '? (0x26B3BBA8)',
 1: '? (0x3EB026B9)',
 2: '? (0x4752DD3A)',
 3: '? (0x7C120F6A)',
 4: '? (0x9A5FEA16)',
 5: '? (0xAAC5A4C0)',
 6: '? (0xEBA95C18)',
 7: '? (0xEE8DADF1)',
 8: '? (0xFAF0B948)',
 9: 'W00005',
 10: 'W00006',
 11: 'W00007',
 12: 'W00008',
 13: 'W00009',
 14: 'W00010',
 15: 'W00011',
 16: 'W00012',
 17: 'W00013',
 18: 'W00014',
 19: 'W00015',
 20: 'W00017',
 21: 'W00018',
 22: 'W00019',
 23: 'W00021',
 24: 'W00024',
 25: 'W00025',
 26: 'W00026',
 27: 'W00027',
 28: 'W00028',
 29: 'W00030',
 30: 'W00031',
 31: 'W00032',
 32: 'W00033',
 33: 'W00035',
 34: 'W00036',
 35: 'W00037',
 36: 'W00038',
 37: 'W00039',
 38: 'W00040',
 39: 'W00041',
 40: 'W00042',
 41: 'W00043',
 42: 'W00044',
 43: 'W00045',
 44: 'W00046',
 45: 'W00048',
 46: 'W00049',
 47: 'W00050',
 48: 'W00051',
 49: 'W00055',
 50: 'W00057',
 51: 'W00058',
 52: 'W00060',
 53: 'W00061',
 54: 'W00062',
 55: 'W00063',
 56: 'W00064',
 57: 'W00065',
 58: 'W00066',
 59: 'W00067',
 60: 'W00068',
 61: 'W00069',
 62: 'W0

In [11]:
df['Bike#'] = bike_labels

In [14]:
x = df.iloc[:,[0,3,5]].values
y = df.iloc[:,-1].values

In [15]:
print(y)

['Registered' 'Casual' 'Registered' ... 'Registered' 'Casual' 'Registered']


In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state = 0)

In [17]:
tree = DecisionTreeClassifier(max_leaf_nodes=3, random_state=0)
tree.fit(x_train,y_train)
y_pred = tree.predict(x_test)

In [20]:
score = metrics.accuracy_score(y_test, y_pred)
print("Accuracy of our model is: {:.1f}%".format(score*100))
print("\nConfusion Matrix-> \n")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report->\n")
print(classification_report(y_test, y_pred))

Accuracy of our model is: 88.4%

Confusion Matrix-> 

[[ 1264  3100]
 [  379 25269]]

Classification Report->

              precision    recall  f1-score   support

      Casual       0.77      0.29      0.42      4364
  Registered       0.89      0.99      0.94     25648

    accuracy                           0.88     30012
   macro avg       0.83      0.64      0.68     30012
weighted avg       0.87      0.88      0.86     30012

